# The Ultimate - Range Distribution

**Please read this so you don't have to read the errors!**  
There is a function which does all things ar once. So you don't have to screatch your head about maintaining the data and so... <br> <br> 
(with added functionality of `Mean` and `Meadian`)
### The function takes:
```python
classify_data(data, 
              mode= 'in',
              precentage= False,
              cf= False,
              cf_type= 'lthan',
              cf_per= False,
              kind= None,
              auto_header= False)
```

Each of the parameters are self explainatory, but you can tweak them as your need.  
Enjoy!

In [1]:
from table import Table
import numpy as np
import math

##### The core of calculation - Class Interval

In [2]:
# Range class created for convenience
class Range:
    '''This `Range` is different from `range` as you can notice.import
    This class manages the Range to be provided during classification of
    the raw ungrouped data to the grouped data.
    
    Give mode = 'in' for an Inclusive range and 'ex' for Exclusive.
    '''

    
    def __init__(self, lower, upper, mode= 'in'):
        self._lower = lower
        self._upper = upper
        self._mode = mode
        self.freq = 0
        self._xi = (self._lower + self._upper) / 2

    @property        
    def fixi(self):
        self._fixi = self._xi * self.freq
        return self._fixi
        
    def __contains__(self, observation):
        if self._mode == 'ex':
            condition = self._lower <= observation < self._upper
        else:
            condition = self._lower <= observation <= self._upper
        if condition:
            return True
        return False

    def __repr__(self):
        return str(f'{self._lower} - {self._upper}')

#### Defining actual function to be called - One function for Everything.

In [3]:
def classify_data(data, mode= 'in', percentage= False, cf= False, cf_type= 'lthan', cf_per= False, kind= None, auto_header= False):
    '''This function will make the arrangement of the data based on the parameters
    you provide.
    Acceptable parameters:
    
    • data: The 1D data that you provide (required)
    • mode: 'in' for inclusive & 'ex' for exclusive
    • precentage: True or False
    • cf: True or False
    • cf_type: 'lthan' for Less Than & 'gthan' for Greter than - provide only if cf = True
    • cf_per: True or False
    • kind: 'mean' or 'median'
    • auto_header: True or False
    
    You can toggle the visibility of your required column by these parameters.
    Enjoy!
    '''

    range_ = max(data) - min(data)
    N = len(data)
    k = math.ceil(1 + 3.322 * (np.log10(range_))) # speager's method
    class_width = math.ceil(range_ / k)

    mode = mode.lower()
    if mode not in ['in', 'ex']:
        raise NotImplementedError(f"The mode must be either 'in' or 'ex'. You provided -> {mode}")
    else:
        offset = 1 if mode == 'in' else 0

    # Range objects
    prev = min(data)
    intervals = []
    for cls in range(k):
        intervals.append(Range(prev, prev + class_width, mode= mode))
        prev += class_width + offset
    
    # Counting freq
    for obs in data:
        for class_ in intervals:
            if obs in class_:
                class_.freq += 1
                break
    
    ' -- Main computation heavy part is done, now handling the parameters -- '
    departure = {} # Where all final features will be stored and will
                   # be shown in the final table.
    

    # Percentage
    if percentage == True:
        departure['percentage'] = []
        for intrvl in intervals:
            prc = round((intrvl.freq * 100) / N, 1)
            departure['percentage'].append(str(prc) + " %")
    
    
    # Function which returns cf
    def get_cf(cf_type):
        cfs = []
        if cf_type == 'lthan':
            prev_freq = 0
            for intrvl in intervals:
                cumulated = prev_freq + intrvl.freq
                cfs.append(cumulated)
                prev_freq += intrvl.freq
        
        elif cf_type == 'gthan':
            prev_freq = 0
            total_t = N
            for intrvl in intervals:
                cumulated = total_t - prev_freq
                cfs.append(cumulated)
                total_t -= prev_freq
                prev_freq = intrvl.freq
        return cfs


    # Cumulative Freq
    if cf == True:
        cf_type = cf_type.lower()
        if cf_type not in ['lthan', 'gthan']:
            raise NotImplementedError(f"The cf_type must be either 'lthan' or 'gthan'. You provided -> {cf_type}")
        else:
            departure['cf'] = get_cf(cf_type)


    # CF percentage
    if cf_per == True and cf == True:
        departure['cf_per'] = []
        for cf in departure['cf']:
            per = round((100 * cf) / N, 1)
            departure['cf_per'].append(str(per) + " %")
    

    # If mean or meadian is to be found
    if kind:
        if kind.lower() == 'mean':
            departure['xi'] = []
            departure['fixi'] = []
            Efixi = 0
            
            for intrvl in intervals:
                departure['xi'].append(intrvl._xi)
                departure['fixi'].append(intrvl.fixi)
                Efixi += intrvl.fixi
            
            central_val = Efixi / N
        
        elif kind.lower() == 'median':
            if cf != True or cf_type == 'gthan':
                departure['cf'] = get_cf('lthan')
            
            th = (N + 1) / 2
            for i, intrvl in enumerate(intervals):
                th_cf = departure['cf'][i]
                if th_cf >= th:
                    m = 0 if i == 0 else departure['cf'][i-1]
                    central_val = intrvl._lower + (((N / 2) - m) / intrvl.freq) * class_width
                    break
         
        else: raise NotImplementedError("Provide either 'mean' or 'median'")

    # Reshaping data to be printed in a tabular format
    reshaped = []
    total = 0
    
    for i, entry in enumerate(intervals): 
        reshaped.append([entry, entry.freq])
        
        for functionality in departure:
            reshaped[-1].append(departure[functionality][i])
        total += entry.freq

    if kind: total = [total, central_val]
    if auto_header:
        headers = ['Classes', 'Frequencies']
        for key in departure:
            headers.append(key)
        return reshaped, total, headers
    return reshaped, total

### 

# That's it!
Let's call the function to make it work. 

In [4]:
# Data
data= np.random.randint(0, 100, 200)

##### Inclusive

In [5]:
cls, total = classify_data(data, mode= 'in')

tbl = Table(cls, headers= ['Classes', 'Frequencies'])
tbl.construct()
print('Total: ', total)

 + ---------- + ------------- + 
 |  Classes   |  Frequencies  | 
 + ---------- + ------------- + 
 |     0 - 13 |            20 | 
 |    14 - 27 |            36 | 
 |    28 - 41 |            28 | 
 |    42 - 55 |            28 | 
 |    56 - 69 |            29 | 
 |    70 - 83 |            28 | 
 |    84 - 97 |            27 | 
 |   98 - 111 |             4 | 
 + ---------- + ------------- + 
Total:  200


##### Exclusive

In [6]:
cls, total = classify_data(data, mode= 'ex')

tbl = Table(cls, headers= ['Classes', 'Frequencies'])
tbl.construct()
print('Total: ', total)

 + ---------- + ------------- + 
 |  Classes   |  Frequencies  | 
 + ---------- + ------------- + 
 |     0 - 13 |            19 | 
 |    13 - 26 |            34 | 
 |    26 - 39 |            25 | 
 |    39 - 52 |            28 | 
 |    52 - 65 |            23 | 
 |    65 - 78 |            29 | 
 |    78 - 91 |            25 | 
 |   91 - 104 |            17 | 
 + ---------- + ------------- + 
Total:  200


##### Other options if you need them

Given `auto_header = True`

In [7]:
cls, total, headers = classify_data(data, mode= 'in', percentage= True, auto_header= True)

tbl = Table(cls, headers= headers)
tbl.construct()
print('Total: ', total)

 + ---------- + ------------- + ------------ + 
 |  Classes   |  Frequencies  |  percentage  | 
 + ---------- + ------------- + ------------ + 
 |     0 - 13 |            20 |       10.0 % | 
 |    14 - 27 |            36 |       18.0 % | 
 |    28 - 41 |            28 |       14.0 % | 
 |    42 - 55 |            28 |       14.0 % | 
 |    56 - 69 |            29 |       14.5 % | 
 |    70 - 83 |            28 |       14.0 % | 
 |    84 - 97 |            27 |       13.5 % | 
 |   98 - 111 |             4 |        2.0 % | 
 + ---------- + ------------- + ------------ + 
Total:  200


# 

#### Try `Row = True`

In [8]:
cls, total, headers = classify_data(data, mode= 'in', percentage= True, auto_header= True)

tbl = Table(cls, headers= headers)
tbl.construct(rows= True)
print('Total: ', total)

 + ---------- + ------------- + ------------ + 
 |  Classes   |  Frequencies  |  percentage  | 
 + ---------- + ------------- + ------------ + 
 |     0 - 13 |            20 |       10.0 % | 
 ---------------------------------------------
 |    14 - 27 |            36 |       18.0 % | 
 ---------------------------------------------
 |    28 - 41 |            28 |       14.0 % | 
 ---------------------------------------------
 |    42 - 55 |            28 |       14.0 % | 
 ---------------------------------------------
 |    56 - 69 |            29 |       14.5 % | 
 ---------------------------------------------
 |    70 - 83 |            28 |       14.0 % | 
 ---------------------------------------------
 |    84 - 97 |            27 |       13.5 % | 
 ---------------------------------------------
 |   98 - 111 |             4 |        2.0 % | 
 + ---------- + ------------- + ------------ + 
Total:  200


#### With Mean
Given `auto_headers = True`

In [9]:
cls, total, headers = classify_data(data, mode= 'in',
                                    kind= 'mean', auto_header= True)

Table(cls, headers= headers).construct(rows= False)
print('Total: ', total[0])
print(' Mean: ', total[1])

 + ---------- + ------------- + ------- + -------- + 
 |  Classes   |  Frequencies  |    xi   |   fixi   | 
 + ---------- + ------------- + ------- + -------- + 
 |     0 - 13 |            20 |     6.5 |    130.0 | 
 |    14 - 27 |            36 |    20.5 |    738.0 | 
 |    28 - 41 |            28 |    34.5 |    966.0 | 
 |    42 - 55 |            28 |    48.5 |   1358.0 | 
 |    56 - 69 |            29 |    62.5 |   1812.5 | 
 |    70 - 83 |            28 |    76.5 |   2142.0 | 
 |    84 - 97 |            27 |    90.5 |   2443.5 | 
 |   98 - 111 |             4 |   104.5 |    418.0 | 
 + ---------- + ------------- + ------- + -------- + 
Total:  200
 Mean:  50.04


#### With Median

In [10]:
cls, total, headers = classify_data(data, mode= 'in',
                                    kind= 'median', auto_header= True)

Table(cls, headers= headers).construct(rows= False)
print(' Total: ', total[0])
print('Median: ', total[1])

 + ---------- + ------------- + ----- + 
 |  Classes   |  Frequencies  |   cf  | 
 + ---------- + ------------- + ----- + 
 |     0 - 13 |            20 |    20 | 
 |    14 - 27 |            36 |    56 | 
 |    28 - 41 |            28 |    84 | 
 |    42 - 55 |            28 |   112 | 
 |    56 - 69 |            29 |   141 | 
 |    70 - 83 |            28 |   169 | 
 |    84 - 97 |            27 |   196 | 
 |   98 - 111 |             4 |   200 | 
 + ---------- + ------------- + ----- + 
 Total:  200
Median:  49.42857142857143


#### Mean and Median will change slightly based on the mode you give i.e. 'Inclusive' or 'Exclusive'

###  

### With all options...

In [11]:
cls, total, headers = classify_data(data, mode= 'in', percentage= True,
                                    cf= True, cf_type= 'gthan', cf_per= True,
                                    kind= 'mean', auto_header= True)

Table(cls, headers= headers).construct(rows= False)
print('Total: ', total[0])
print(' Mean: ', total[1])

 + ---------- + ------------- + ------------ + ----- + --------- + ------- + -------- + 
 |  Classes   |  Frequencies  |  percentage  |   cf  |   cf_per  |    xi   |   fixi   | 
 + ---------- + ------------- + ------------ + ----- + --------- + ------- + -------- + 
 |     0 - 13 |            20 |       10.0 % |   200 |   100.0 % |     6.5 |    130.0 | 
 |    14 - 27 |            36 |       18.0 % |   180 |    90.0 % |    20.5 |    738.0 | 
 |    28 - 41 |            28 |       14.0 % |   144 |    72.0 % |    34.5 |    966.0 | 
 |    42 - 55 |            28 |       14.0 % |   116 |    58.0 % |    48.5 |   1358.0 | 
 |    56 - 69 |            29 |       14.5 % |    88 |    44.0 % |    62.5 |   1812.5 | 
 |    70 - 83 |            28 |       14.0 % |    59 |    29.5 % |    76.5 |   2142.0 | 
 |    84 - 97 |            27 |       13.5 % |    31 |    15.5 % |    90.5 |   2443.5 | 
 |   98 - 111 |             4 |        2.0 % |     4 |     2.0 % |   104.5 |    418.0 | 
 + ---------- + -----

###  

Try out all combinations of the parameters based on your convenience.

###  

###  

## Time Complexity

Have tried the algorithm **WITH ALL PARAMETERS: turend ON**  and we get the following optimized time complexity

##### With 1,000 samples

In [15]:
data= np.random.randint(0, 100, 1000)
%timeit classify_data(data, mode= 'in', percentage= True, cf= True, cf_type= 'gthan', cf_per= True, kind= 'mean', auto_header= True)

1.5 ms ± 5.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


##### With 10,000 samples

In [16]:
data= np.random.randint(0, 100, 10000)
%timeit classify_data(data, mode= 'in', percentage= True, cf= True, cf_type= 'gthan', cf_per= True, kind= 'mean', auto_header= True)

14.7 ms ± 52.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


##### With 10,00,000 samples

In [17]:
data= np.random.randint(0, 100, 10_00_000)
%timeit classify_data(data, mode= 'in', percentage= True, cf= True, cf_type= 'gthan', cf_per= True, kind= 'mean', auto_header= True)

1.46 s ± 9.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


###  

You can try out more and with more samples. I came up with this the most optimized version.  
Of course it can be further optimized somewhere... but I found it "works well" for me type.

**EDIT**: Have just realized that, this version of code works surprisingly faster than the   
past one. (Also in past there were no mean or median - In current version mean and meadian are there!)

# Thanks!